# Notebook activity B2.3 Distributed computing in R

In this activity, we will go through using distributed (parallel) computing within R. For this, we will make use of the following R package: `parallel`. Recall that packages extend the capability of R by providing extra functions and datasets. You have already been making use of one such package during your study of M348, a package called `M348`. This has provided datasets and functions used in the notebook activities supplied with M348. 

When using packages in R, there are two stages. First, the package needs to be installed. This has already been done as part of the set-up of the VCE. Then, every time you want to make use of the package in R, you have to tell R to load it. You will have got used to doing this for the package `M348` by using the command `library(M348)`. The equivalent command for the `parallel` package is given by `library(parallel)`. 

## (a) Getting started

### (i)

Using the code cell below, load the package `parallel`. Note that if you have done this step correctly, you should not get any output. (But it will tell you if there is an error.)

#### Solution <a class="tocSkip">

In [1]:
library(parallel)

###  <a class="tocSkip">

Although this activity is about distributed computing, you will still only be making use of your computer. However, most computers these days have multiple 'cores' or 'processors'. We will be making use of these cores in this activity. 

### (ii)

The number of cores available to R varies from computer to computer. Check how many cores are available to you now by running the following command.

In [ ]:
detectCores()

#### Solution <a class="tocSkip">

The output should be an integer which indicates the number of cores in process for your computer. 

---

###  <a class="tocSkip">

In the following, we will assume that the number of cores available to you is more than 1. (If you have only one core, your computer will not be able to do computations in parallel.)

## (b) Re-creating the function from Notebook activity B2.2

In this notebook activity, we will focus on fitting a simple linear regression model using parallel computing. As you learned when you worked through the notebook activities asscociated with Unit 1, R has a function for fitting a simple linear regression model (and linear models in general): `lm()`. However, in this notebook activity, you will be working with the function you created in Notebook activity B2.2. This will make it easier for you to switch to using parallel computing.

### (i)

As this is a different notebook, you need to run the code to create `mylm(x, y)` again. So, copy the code across from part (b)(iii) of Notebook B2.2 and run it. 

#### Solution <a class="tocSkip">

The required code is given below. Running this code will produce no output.

In [3]:
mylm <- function(x, y){
    
# Step 1
sumX <- sum(x)
sumY <- sum(y)
sumXX <- sum(x ^ 2)
sumXY <- sum(x * y)

# Step 2
n <- length(x)
xBar <- sumX / n
yBar <- sumY / n

# Step 3
sXX <- sumXX - n * xBar ^ 2
sXY <- sumXY - n * xBar * yBar

# Step 4
betaHat <- sXY/sXX

# Step 5
alphaHat <- yBar - betaHat * xBar

# Producing the output
cat("alphaHat is ", alphaHat, " and betaHat is ", betaHat,".\n", sep="")

}

### (ii)

Re-test your function by running the following code. If you have correctly defined the `mylm()` function, running the code below should output `alphaHat` and `betaHat`.

In [ ]:
w <- c(2, 4, 6, 8)
z <- c(14, 18, 22, 26)

mylm(w, z)

###  <a class="tocSkip">

So far, you have only been applying `mylm(x, y)` to very small datasets. To demonstrate the impact of parallel computing we will need to work with much larger datasets. 

We will work with simulated data in order to easily control the size of the dataset. 

## (c) Working with much larger datasets

The following example code creates two vectors `x` and `y` each containing `n` values.
```
n <- 1000
x <- 10 * c(1:n) / n
y <- 10 + 2 * x + rnorm(n)
```
Note that the function `rnorm(n)` creates `n` values from the standard normal distribution. So, these data fit the following simple linear regression model
$$y_i = 10 + 2x_i + W_i,$$
where $W_i \sim N(0, 1)$.

### (i)

In the code cell below, run the commands given above and use `mylm(x, y)` to fit a simple linear regression. Check that the estimates seem sensible.

#### Solution <a class="tocSkip">

The required code is given below. Running the code will output `alphaHat` and `betaHat`. These estimates should be sensible in the sense that `alphaHat` should be close to 10 and `betaHat` should be close to 2.

In [5]:
n <- 1000
x <- 10 * c(1:n) / n
y <- 10 + 2 * x + rnorm(n)

mylm(x, y)

alphaHat is 10.07484 and betaHat is 1.981297.


## (d) Doing distributed computing in R

So far, R has not been using distributed computing to do the calculations in `mylm()`. To do this, we need to create a version of this function, `mylmParallel()`, that does enable R to use distributed computing. This has been done for you below. 

### (i)

Run the following code to create the `mylmParallel()` function.

In [6]:
mylmParallel <- function(x, y, ncl){
    
# Implementing the distributed computing
    xSplit <- split(x, 1:ncl)
    ySplit <- split(y, 1:ncl)
    cl <- makeCluster(ncl)
    clusterExport(cl, c("xSplit", "ySplit"), envir = environment())

    sumX <- parLapply(cl, xSplit, sum)
    sumY <- parLapply(cl, ySplit, sum)
    xySplit <- clusterMap(cl, '*', xSplit, ySplit, SIMPLIFY=FALSE)
    xxSplit <- clusterMap(cl, '*', xSplit, xSplit, SIMPLIFY=FALSE)

    stopCluster(cl)

# Step 1
    sumX <- sum(unlist(xSplit))
    sumY <- sum(unlist(ySplit))
    sumXX <- sum(unlist(xxSplit))
    sumXY <- sum(unlist(xySplit))

# Step 2
    n <- length(x)
    xBar <- sumX / n
    yBar <- sumY / n

# Step 3
    sXX <- sumXX - n * xBar ^ 2
    sXY <- sumXY - n * xBar * yBar

# Step 4
    betaHat <- sXY / sXX

# Step 5
    alphaHat <- yBar - betaHat * xBar

# Producing the output
    cat("alphaHat is ", alphaHat, " and betaHat is ", betaHat,".\n", sep="")
    
}


### (ii)

Take a look at the definition of the `mylmParallel()` function in part (i). How have the arguments for this function changed?

#### Solution <a class="tocSkip">

The function `mylmParallel()` has gained an extra argument: `ncl`. This corresponds to the number of cores that will be used for the computations. Below, we will set `ncl` to be equal to 4. This can be any number less than or equal to the number given by `detectCores()`.

### (iii)

Now look at the code within the `mylmParallel()` function. In what way has it changed from `mylm()`?

#### Solution <a class="tocSkip">

Only the code with Step 1 has changed. The most obvious thing to note is that it has become more complicated. There is extra code needed to set up the cores, and to make the right data available to them (and in the right form). Also, though you might have noticed that the function `sum()` is still being used, it is done in a more complicated way. 

## (e) Checking computational time

With distributed computing, we hope that it will speed up computations. So, we need a way of assessing how quickly R completes a computation.

In principle, we could ask you to get out a stopwatch and do the timing manually. But the times involved are sufficiently small (we hope!) that this is impractical. Instead, we will make use of the R function: `Sys.time()`. This function saves the current time, to fractions of a second, to be stored in an object. The length of time taken to complete a computation can therefore be obtained by comparing the recorded time just before starting the computation and just afterwards. 

### (i)

Use the following commands to time how long the function `mylm(x, y)` takes when there are 1000 datapoints.

In [ ]:
startTime <- Sys.time()
mylm(x, y)
finishTime <- Sys.time()
cat("Time taken is ", finishTime - startTime," seconds.\n", sep="")

### (ii) 

Now by adapting the commands in part (i), time how long the function `mylmParallel(x, y, 4)` takes when there are 1000 datapoints.

#### Solution <a class="tocSkip">

The required code is given below. Running the code will output `alphaHat`, `betaHat` and the computational time.

In [8]:
startTime <- Sys.time()
mylmParallel(x, y, 4)
finishTime <- Sys.time()
cat("Time taken is ", finishTime - startTime," seconds.\n", sep="")

alphaHat is 10.07484 and betaHat is 1.981297.
Time taken is 1.204897 seconds.


### (iii)

Compare the two timings in parts (i) and (ii). Which function was faster?

#### Solution <a class="tocSkip">

This timings will vary from computer to computer - and even vary a bit every time you try it. However, when a member of the module team tried this on their computer, they found that `mylm()` took about 0.004 seconds, whereas `mylmParallel()` took just over half a second. So, using distributed computing was much slower!

## (f) Finishing off

You probably found that using parallel computing in this example has actually slowed the computation down. This is not surprising with small datasets. To ensure that the computations in the different cores all come together properly, they require careful handling. Whilst using a package, such as `parallel`, shields us from having to deal with the details of that, it translates into an overhead in terms of computational time. The overhead is large enough that for "small" datasets it will not be worth it. 

This, of course, leads to the question of how big a dataset needs to be before distributed computing is worth it. Unfortunately, there is not a rule of thumb for this as it depends on what is being done with the data, as well as how much of data is available. However, a good indicator that distributed computing might be useful is when computations take too long otherwise. 